Koden under er en versjon som kjører synkront.

In [ ]:
import time

def make_meal(delay, what):
    print(what, end=": ")
    time.sleep(delay)
    print('Ferdig!')

def main():
    print(f"Startet {time.strftime('%X')}")
    
    make_meal(3, 'Lage kaffe')
    make_meal(5, 'Koke egg')
    make_meal(3, 'Steike bacon')
    make_meal(5, 'Varme bønner i tomatsaus')
    make_meal(3, 'Riste brød')
    make_meal(1, 'Sette på bordet')

    print('Frokosten er ferdig!')
    print(f"Ferdig ved {time.strftime('%X')}")

main()

Og nå benytter vi oss av asyncio for å kunne jobbe asynkront med jobber som kan gjøres i parallell

In [ ]:
import asyncio
import time

async def make_meal(delay, what):
    await asyncio.sleep(delay)
    print(what, ": Ferdig!")

async def main() :

    print(f"Startet {time.strftime('%X')}")
    await asyncio.gather(make_meal(3, 'Lage kaffe'),
        make_meal(5, 'Koke egg'),
        make_meal(3, 'Steike bacon'),
        make_meal(5, 'Varme bønner i tomatsaus'),
        make_meal(3, 'Riste brød'),
        make_meal(1, 'Sette på bordet'))
    print(f"Ferdig ved {time.strftime('%X')}")

await main()


Skal man utveksle informasjon på tvers av jobber er det naturlig å gjøre dette via en kø. Det finnes tråd-sikre måter å gjøre dette på.

In [33]:
import asyncio
import random
import time


async def worker(name, queue):
    while True:
        # Get a "work item" out of the queue.
        sleep_for = await queue.get()

        # Sleep for the "sleep_for" seconds.
        await asyncio.sleep(sleep_for)

        # Notify the queue that the "work item" has been processed.
        queue.task_done()

        print(f'{name} has slept for {sleep_for:.2f} seconds')


async def main():
    # Create a queue that we will use to store our "workload".
    queue = asyncio.Queue()

    # Generate random timings and put them into the queue.
    total_sleep_time = 0
    for _ in range(20):
        sleep_for = random.uniform(0.05, 1.0)
        total_sleep_time += sleep_for
        queue.put_nowait(sleep_for)

    # Create three worker tasks to process the queue concurrently.
    tasks = []
    for i in range(3):
        task = asyncio.create_task(worker(f'worker-{i}', queue))
        tasks.append(task)

    # Wait until the queue is fully processed.
    started_at = time.monotonic()
    await queue.join()
    total_slept_for = time.monotonic() - started_at

    # Cancel our worker tasks.
    for task in tasks:
        task.cancel()
    # Wait until all worker tasks are cancelled.
    await asyncio.gather(*tasks, return_exceptions=True)

    print('====')
    print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
    print(f'total expected sleep time: {total_sleep_time:.2f} seconds')


await main()

worker-0 has slept for 0.17 seconds
worker-2 has slept for 0.67 seconds
worker-1 has slept for 0.84 seconds
worker-1 has slept for 0.21 seconds
worker-0 has slept for 0.90 seconds
worker-2 has slept for 0.52 seconds
worker-1 has slept for 0.16 seconds
worker-0 has slept for 0.46 seconds
worker-1 has slept for 0.39 seconds
worker-2 has slept for 0.59 seconds
worker-2 has slept for 0.34 seconds
worker-2 has slept for 0.13 seconds
worker-1 has slept for 0.77 seconds
worker-0 has slept for 0.94 seconds
worker-1 has slept for 0.09 seconds
worker-2 has slept for 0.39 seconds
worker-1 has slept for 0.47 seconds
worker-2 has slept for 0.26 seconds
worker-0 has slept for 0.99 seconds
worker-1 has slept for 0.67 seconds
====
3 workers slept in parallel for 3.64 seconds
total expected sleep time: 9.96 seconds
